In [2]:
from selenium import webdriver
import string
import pandas as pd
import bs4 as bs
import os
import numpy as np
import re
import time
from selenium.webdriver.common.keys import Keys
import requests
from selenium.common.exceptions import NoSuchElementException

In [3]:
driver = webdriver.Chrome()
driver.get("http://proxy.its.virginia.edu/login?url=http://heinonline.org/HOL/Welcome")
l_driver = webdriver.Chrome()
l_driver.get('https://catalog.loc.gov/vwebv/searchBrowse')
g_driver = webdriver.Chrome()

In [4]:
def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

In [5]:
username = 'meb2fv'
password = '0ver In 2018!'
driver.find_element_by_id("user").send_keys(username);
driver.find_element_by_id("pass").send_keys(password);
driver.find_element_by_xpath("/html/body/main/div[2]/fieldset/form/input").click()

In [6]:
def search_names(mid_first_name, last_name, school_url):
    link = 'https://heinonline-org.proxy01.its.virginia.edu/HOL/LuceneSearch?typea=title&termsa=&operator=AND&typeb=creator&termsb=' + last_name + '+' + mid_first_name + '&operatorb=AND&typec=text&termsc=&operatorc=AND&typed=title&termsd=&operatord=AND&typee=title&termse=&operatore=AND&typef=title&termsf=&yearlo=&yearhi=&tabfrom=&searchtype=field&collection=all&submit=Go'
    driver.get(link)
    try:
        webpage_wait('//*[@id="heinlogo"]/a/img')
        driver.find_element_by_xpath('//*[@id="search_modify"]/form/div/div/div/div/a[4]/i').click()
    except:
        driver.find_element_by_xpath('//*[@id="search_modify"]/div')
    element = driver.find_elements_by_tag_name('a')
    full_name = mid_first_name + ' ' +  last_name
    alt_fm_names = []
    err_fm_names = []
    if ' ' in mid_first_name.lower():
        first_name = mid_first_name.split(' ')[0]
    else: 
        first_name = mid_first_name
    page = 1
    
    while element:
        for link in element:
            link_text = link.text.lower()
            if first_name.lower() in link_text.lower() and last_name.lower() in link_text.lower() and '[' not in link_text:
                try:
                    new_last = link_text.split(', ')[0]
                    new_first_mid = link_text.split(', ')[1]
                    if first_name.lower() in new_first_mid and last_name.lower() == new_last:
                        new_fm = link.text.split(', ')[1]
                        if not new_fm in alt_fm_names and not new_fm in err_fm_names:
                            faculty = check_google(new_fm, last_name, school_url)
                            if faculty: 
                                alt_fm_names.append(new_fm)
                            else: 
                                err_fm_names.append(new_fm)
                except:
                    continue
        if page < 2:
            try:
                driver.find_element_by_xpath('//*[@id="thenext"]/span').click()
                time.sleep(3)
                element = driver.find_elements_by_tag_name('a') 
                page += 1
            except:
                element = []
        else: 
            element = []
    return alt_fm_names, err_fm_names

In [7]:
def check_google(mid_first_name, last_name, school_url):
    faculty = False
    for url in school_url:
        if faculty == True:
            break
        g_driver.get("http://google.com")
        search = g_driver.find_element_by_name('q')
        if not ' ' in mid_first_name:
            search.send_keys(mid_first_name + ' ' + last_name + ' ' + url)
        else: 
            search.send_keys(mid_first_name + ' ' + last_name + ' ')
        search.send_keys(Keys.RETURN)
        elems = g_driver.find_elements_by_xpath("//a[@href]")

        for elem in elems:
            if url in elem.text:
                print(url + ' in: ' + elem.text)
                faculty = True
                break
    return faculty

In [8]:
def search_loc(fm_name, last_name):
    l_driver.get('https://catalog.loc.gov/vwebv/searchBrowse')
    name = last_name + ', ' + fm_name
    l_driver.find_element_by_xpath('//*[@id="search-code"]').click()
    l_driver.find_element_by_xpath('//*[@id="search-code"]/option[2]').click()
    l_driver.find_element_by_xpath('//*[@id="search-code"]').click()
    search = l_driver.find_element_by_xpath('//*[@id="search-argument"]')
    search.send_keys(name)
    l_driver.find_element_by_xpath('//*[@id="search-button"]').click()

In [9]:
def scrape_titles(name):
    title_list = []
    i = 1
    try:
        book_element = l_driver.find_element_by_xpath('//*[@id="search-results"]/ul/li[' + str(i) + ']/div[3]/div/div/a')
    except NoSuchElementException as exception:
        book_element = []
    while book_element:
        try:
            name_element = l_driver.find_element_by_xpath('//*[@id="search-results"]/ul/li[' + str(i) + ']/div[3]/div/div[2]')
            author_name = name_element.text
            if name in author_name:
                book_title = book_element.text.split(' /', 1)[0]
                if book_title not in title_list:
                    title_list.append(book_title)
        except NoSuchElementException as exception:
            print('no such element')
        i += 1
        try:
            book_element = l_driver.find_element_by_xpath('//*[@id="search-results"]/ul/li[' + str(i) + ']/div[3]/div/div[3]/a')
        except NoSuchElementException as exception:
            book_element = []  
    return title_list

In [12]:
main_df = pd.DataFrame(columns = ['Name', 'School','Books'])
path = create_path('school_data')
files = os.listdir(path)
url_list = ['asu', 'bc', 'bu', 'columbia', 'cornell']
for j in range(len(files)): 
    file = files[j]
    data = pd.read_csv(create_path('school_data', file))
    for i in range(len(data)):
        #Get name and information from the database
        fm_name = data['First Name'][i]
        last_name = data['Last Name'][i]
        alt_names, err_names = search_names(fm_name, last_name, [url_list[j]])
        alt_names
        for fm_name in alt_names:
            print(fm_name)
            search_loc(fm_name, last_name)
            try:
                first_result = l_driver.find_element_by_xpath('//*[@id="heading-0"]')
                print(first_result)
                full_name = last_name + ', ' + fm_name
                print(full_name)
                title_list = 'na'
                if first_result.text.lower().translate(str.maketrans('', '', string.punctuation)) == full_name.lower().translate(str.maketrans('', '', string.punctuation)):
                    first_result.click()
                    title_list = scrape_titles(full_name)
                    if not title_list:
                        title_list = 'na'
                main_df = main_df.append(pd.DataFrame({'Name': [fm_name + ' ' +  last_name], 'School': [file], 'Books': [title_list]}))
                print(title_list)
            except NoSuchElementException as exception:
                continue

asu in: Kenneth Abbott - Person - Julie Ann Wrigley Global Institute of ...
https://sustainability.asu.edu/person/kenneth-abbott
Kenneth W.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.7255865758387978-1")>
Abbott, Kenneth W.
na
asu in: Angela Banks | iSearch - ASU Faculty and Staff Search
https://isearch.asu.edu/profile/3153941
asu in: Angela Banks | iSearch
https://isearch.asu.edu/profile/3153941
Angela
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.7274268724502548-1")>
Banks, Angela
na
Angela M.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.2254046742495608-1")>
Banks, Angela M.
na
asu in: Paul Bender | iSearch - ASU Faculty and Staff Search
https://isearch.asu.edu/profile/274462
Paul
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.2871630390685178-1")

asu in: Zachary Gubler | iSearch
https://isearch.asu.edu/profile/1803801
Zachary J.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.6680745711545524-1")>
Gubler, Zachary J.
na
asu in: Tamara Herrera | iSearch - ASU Faculty and Staff Search
https://isearch.asu.edu/profile/406622
Tamara
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.5556582440763418-1")>
Herrera, Tamara
na
asu in: Art Hinshaw | iSearch - ASU Faculty and Staff Search
https://isearch.asu.edu/profile/733466
Art
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.4236288763092684-1")>
Hinshaw, Art
na
asu in: James Hodge (Law College) | iSearch
https://isearch.asu.edu/profile/1436144
James G. Jr.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.03228366957561124-1")>
Hodge, James G. Jr.
na
asu in: Kimberly

James
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.642247158049136-1")>
Weinstein, James
na
bc in: Filippa Anzalone - Law School - Boston College
https://www.bc.edu/bc-web/schools/law/academics-faculty/.../filippa-anzalone.html
Filippa Marullo
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.5824546063455873-1")>
Anzalone, Filippa Marullo
na
bc in: Hugh Ault - Law School - Boston College
https://www.bc.edu/bc-web/schools/law/academics-faculty/faculty.../hugh-ault.html
Hugh J.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.9008996081249709-1")>
Ault, Hugh J.
['Comparative income taxation : a structural analysis']
bc in: Paulo Barrozo - Law School - Boston College
https://www.bc.edu/bc-web/schools/law/academics-faculty/.../paulo-barrozo.html
Paulo
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4

['All-american aphrodite', 'Voice of Peking: "The road to Paris ..." edited by Daniel Lyons and Stephen C. Y. Pan.']
Daniel A.
bc in: Ray Madoff - Law School - Boston College
https://www.bc.edu/bc-web/schools/law/academics-faculty/faculty.../ray-madoff.html
bc in: Ray Madoff - Law School - Boston College
https://www.bc.edu/bc-web/schools/law/academics-faculty/faculty.../ray-madoff.html
Ray
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.1839016691424984-1")>
Madoff, Ray
na
Ray D.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.6479680417598528-1")>
Madoff, Ray D.
['Immortality and the law : the rising power of the American dead', 'Practical guide to estate planning']
bc in: Patricia McCoy - Law School - Boston College
https://www.bc.edu/bc-web/schools/law/academics-faculty/.../patricia-mccoy.html
Patricia A.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a

bu in: George Annas » SPH | Boston University
https://www.bu.edu/sph/profile/george-annas/
George
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.27399082838274347-1")>
Annas, George
na
George J.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.24563539653026756-1")>
Annas, George J.
na
bu in: Hugh W. Baxter | School of Law - Boston University
https://www.bu.edu/law/profile/hugh-w-baxter/
Hugh
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.025267837473542665-1")>
Baxter, Hugh
na
bu in: Jack M. Beermann | School of Law - Boston University
https://www.bu.edu/law/profile/jack-m-beermann/
bu in: Jack M. Beermann | School of Law - Boston University
https://www.bu.edu/law/profile/jack-m-beermann/
Jack
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.3414308987269665-1"

...

columbia in: Zohar Goshen | Columbia Law School
https://www.law.columbia.edu/faculty/zohar-goshen
Zohar
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.13457277381925237-1")>
Goshen, Zohar
na
columbia in: Michael Graetz | Columbia Law School
https://www.law.columbia.edu/faculty/michael-graetz
columbia in: Michael Graetz | Columbia Law School
https://www.law.columbia.edu/faculty/michael-graetz
Michael
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.6652833335166513-1")>
Graetz, Michael
na
Michael J.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.2009644059462059-1")>
Graetz, Michael J.
na
columbia in: Kent Greenawalt | Columbia Law School
https://www.law.columbia.edu/faculty/kent-greenawalt
R. Kent
columbia in: Jamal Greene | Columbia Law School
https://www.law.columbia.edu/faculty/jamal-greene
Jamal
<selenium.webdriv

na
columbia in: Justin McCrary | Columbia Law School
https://www.law.columbia.edu/faculty/justin-mccrary
Justin
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.4125286216976174-1")>
McCrary, Justin
['Effect of court-ordered hiring quotas on the composition and quality of police [electronic resource]', 'Effect of female education on fertility and infant health [electronic resource] : evidence from school entry policies using exact date of birth', 'Manipulation of the running variable in the regression discontinuity design [electronic resource] : a density test']
columbia in: Thomas Merrill | Columbia Law School
https://www.law.columbia.edu/faculty/thomas-merrill
Thomas W.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.013714673385978093-1")>
Merrill, Thomas W.
na
columbia in: Gillian Metzger | Columbia Law School
https://www.law.columbia.edu/faculty/gillian-metzger
columbia

na
columbia in: Eric Talley | Columbia Law School
https://www.law.columbia.edu/faculty/eric-talley
columbia in: Eric Talley | Columbia Law School
https://www.law.columbia.edu/faculty/eric-talley
Eric
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.011723257078849247-1")>
Talley, Eric
na
Eric L.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.97754999656957-1")>
Talley, Eric L.
na
columbia in: Kendall Thomas | Columbia Law School
https://www.law.columbia.edu/faculty/kendall-thomas
Kendall
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.27843045641053954-1")>
Thomas, Kendall
na
columbia in: Matthew Waxman | Columbia Law School
https://www.law.columbia.edu/faculty/matthew-waxman
columbia in: Matthew Waxman | Columbia Law School
https://www.law.columbia.edu/faculty/matthew-waxman
Matthew
<selenium.webdriver.remote.webelem

cornell in: Robert Hillman - Cornell Law School - Cornell University
https://www.lawschool.cornell.edu/faculty/bio_Robert_Hillman.cfm
cornell in: Robert Hillman - Cornell Law School - Cornell University
https://www.lawschool.cornell.edu/faculty/bio_Robert_Hillman.cfm
Robert
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.9990840802564416-1")>
Hillman, Robert
na
Robert A.
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.38864601339338845-1")>
Hillman, Robert A.
na
cornell in: Robert C. Hockett - Cornell Law School - Cornell University
https://www.lawschool.cornell.edu/faculty/bio_robert_hockett.cfm
Robert
<selenium.webdriver.remote.webelement.WebElement (session="9c8753a41b4a5c5f0bc8b69d463c4924", element="0.4015771467074354-1")>
Hockett, Robert
na
cornell in: Barbara J. Holden-Smith - Cornell Law School - Cornell University
https://www.lawschool.cornell.edu/faculty/bio_barba

IndexError: list index out of range

In [14]:
main_df.to_csv('test_book_names.csv')


In [8]:
def search_google_scholar(article_name):
    driver.get('https://scholar.google.com/')
    search = driver.find_element_by_xpath('//*[@id="gs_hdr_tsi"]')
    search.send_keys('allintitle: ' + article_name)
    driver.find_element_by_xpath('//*[@id="gs_hdr_tsb"]/span/span[1]').click()
#     citations = driver.find_element_by_xpath('//*[@id="gs_res_ccl_mid"]/div[1]/div[2]/div[3]/a[3]').text
    try:
        citations = driver.find_element_by_xpath('//*[@id="gs_res_ccl_mid"]/div/div/div[3]/a[3]').text
        try:
            citations = citations.split('by ')[1]
        except IndexError as exception:
            citations = '0'
    except NoSuchElementException as exception:
        print('No article found')
        citations = 'na'
    return citations

In [35]:
os.getcwd()
path = os.path.join(os.getcwd(), 'author_papers', 'Arizona')
files = os.listdir(path)
main_df = pd.DataFrame(columns = ['Name', 'Article', 'Hein Citations', 'Google Citations'])
for file in files[0:1]:
    data = pd.read_csv(os.path.join(path, file))
    name = file.split('_')[0]
    for i in range(0, len(data)):
        article_name = data['Title'][i].split('[')[0]
        print(article_name)
        hein_citations = data['Cited (articles)'][i]
        print(hein_citations)
        citation_number = search_google_scholar(article_name)
        print(citation_number)
        main_df = main_df.append(pd.DataFrame({'Name': [name], 'Article': [article_name], 'Hein Citations': [hein_citations], 'Google Citations': [citation_number]}))
        driver.get('http://amazon.com')
        time.sleep(3)
        driver.get('http://facebook.com')
        time.sleep(3)

Reinventing the Double Helix: A Novel and Nonobvious Reconceptualization of the Biotechnology Patent 
115
229
Civil and Criminal Sanctions in the Constitution and Courts 
58
88
Don't Judge a Sale by Its License: Software Transfers under the First Sale Doctrine in the United States and the European Community 
32
57
Copyright Misuse and the Limits of the Intellectual Property Monopoly 
28
63
State Regulation of Sexuality in International Human Rights Law and Theory 
26
69
Secrecy, Monopoly, and Access to Pharmaceuticals in International Trade Law: Protection of Marketing Approval Data under the TRIPs Agreement 
20
154
U.S. State Legislation to Limit Use of International and Foreign Law 
14
No article found
na
Questioning Civilian Immunity 
13
No article found
na
Challenges and Implications of a Systemic Social Effect Theory 
10
855
Introductory Note to the Anti-Counterfeiting Trade Agreement 
10
No article found
na
A Divorce Waiting to Happen: Franklin Roosevelt and the Law of Neutrality

In [42]:
main_df.to_csv(create_path('google_scholar_citations', 'arizona.csv'))

In [28]:
data

,Unnamed: 0,Title,Author(s),Journal,Cited (articles),Cited (cases),Accessed,Topics
0,0,Reinventing the Double Helix: A Novel and Nono...,"Demaine, Linda J. (Cited 262 times); Fellmeth,...","Stanford Law Review, Vol. 55, Issue 2 (Novembe...",115,na,8,"Intellectual Property Law, Science and Techno..."
1,0,Civil and Criminal Sanctions in the Constituti...,"Fellmeth, Aaron Xavier","Georgetown Law Journal, Vol. 94, Issue 1 (Nove...",58,3,22,"Jurisprudence, Practice and Procedure, Crimin..."
2,0,Don't Judge a Sale by Its License: Software Tr...,"Determann, Lothar (Cited 104 times); Fellmeth,...","University of San Francisco Law Review, Vol. 3...",32,na,13,"Science and Technology, Computer Law, Intelle..."
3,0,Copyright Misuse and the Limits of the Intelle...,"Fellmeth, Aaron Xavier","Journal of Intellectual Property Law, Vol. 6, ...",28,na,15,"Torts, Intellectual Property Law, Trade Regul..."
4,0,State Regulation of Sexuality in International...,"Fellmeth, Aaron Xavier","William and Mary Law Review, Vol. 50, Issue 3 ...",26,na,85,"Education, Fair Credit, Fair Employment, Fair..."
5,0,"Secrecy, Monopoly, and Access to Pharmaceutica...","Fellmeth, Aaron Xavier","Harvard International Law Journal, Vol. 45, Is...",20,na,39,"Food and Drug, Intellectual Property Law, Int..."
6,0,U.S. State Legislation to Limit Use of Interna...,"Fellmeth, Aaron (Cited 376 times)","American Journal of International Law, Vol. 10...",14,na,11,"Legislation, Comparative Law, Terrorism"
7,0,Questioning Civilian Immunity [article],"Fellmeth, Aaron Xavier","Texas International Law Journal, Vol. 43, Issu...",13,na,9,"Crimes Against Humanity, Military, War and Pe..."
8,0,Challenges and Implications of a Systemic Soci...,"Fellmeth, Aaron Xavier","University of Illinois Law Review, Vol. 2006, ...",10,1,1,"Jurisprudence, Practice and Procedure, Crimin..."
9,0,Introductory Note to the Anti-Counterfeiting T...,"Fellmeth, Aaron X.","International Legal Materials, Vol. 50, Issue ...",10,na,5,"Intellectual Property Law, International Trad..."


In [81]:
def split_name(full_name):
    space_count = full_name.count(' ')
    if space_count == 1:
        f_name = full_name.split(' ')[0] 
        last_name = full_name.split(' ')[1]
    elif space_count == 2:
        f_name = full_name.split(' ')[0] 
#                         m_name = full_name.split(' ')[1]
#         fm_name = f_name + ' ' + m_name
        last_name = full_name.split(' ')[2]
    elif space_count > 2:
        f_name = full_name.split(' ')[0] 
#                         m_name = full_name.split(' ')[1]
#         fm_name = f_name + ' ' + m_name
        last_name = ' '.join(full_name.split(' ')[2:])
    return f_name, last_name

In [88]:
def get_links(first_name, last_name, school_url, school_name):
    driver.get("https://scholar.google.com/citations?view_op=search_authors&mauthors=" +  first_name + '+' + last_name + "&hl=en&oi=ao")
    i = 1
    try:
        print(i)
        element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3')
    except:
        element = []
    while element:
        full_name = element.text
        print(full_name)
        f_name, last_name = split_name(full_name)
        if last_name in full_name and f_name in full_name:
            affiliation = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/div[1]')
            print(affiliation.text.lower())
            if (school_name in affiliation.text.lower() or school_url in affiliation.text.lower()) and full_name not in correct_names:
                link_elm = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3/a')
                correct_name = (full_name, link_elm.get_attribute('href'))
                return correct_name
#             else:
#                 faculty = check_google(f_name, last_name, [school_url])
#                 if faculty and full_name not in correct_names:
#                     link_elm = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3/a')
#                     correct_name = (full_name, link_elm.get_attribute('href'))
#                     return correct_name
        i += 1
        try:
            element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3')
        except:
            element = []


In [78]:
# element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[' + str(i) + ']/div/div/h3')
element = driver.find_element_by_xpath('//*[@id="gsc_sa_ccl"]/div[1]/div/div/h3')
element.text

'Kenneth W Abbott'

In [68]:
def create_path(*args):
    cur_path = os.getcwd()
    for value in args:
        cur_path  = os.path.join(cur_path, value)
    return cur_path

In [94]:
os.getcwd()
path = os.path.join(os.getcwd(), 'school_data')
files = os.listdir(path)
school_name = 'arizona'
school_url = 'asu'
for file in files[0:1]:
    main_df = pd.DataFrame(columns = ['Name', 'School', 'Link'])
    data = pd.read_csv(os.path.join(os.getcwd(), 'school_data', file))
    for i in range(len(data)):
        got_page = False
        mid_first_name = data['First Name'][i]
        last_name = data['Last Name'][i]
        full_name = mid_first_name + ' ' +  last_name
        school = data['School'][i]
        link = get_links(mid_first_name, last_name, school_url, school_name)
        print(link)
        driver.get('http://amazon.com')
        time.sleep(3)
        driver.get('http://facebook.com')
        time.sleep(3)
        if link:
            main_df = main_df.append(pd.DataFrame({'Name': [full_name], 'School': [school],'Link': [link[1]]}))
        else:
            main_df = main_df.append(pd.DataFrame({'Name': [full_name], 'School': [school],'Link': ['NA']}))

1
Kenneth W Abbott
jack e brown chair in law emeritus, professor of global studies, arizona state university
('Kenneth W Abbott', 'https://scholar.google.com/citations?hl=en&user=hva7aGkAAAAJ')
1
Angela D Banks
university of san francisco
Angela Abbate
economist, swiss national bank
None
1
None
1
Daniel Bodansky
arizona state university
('Daniel Bodansky', 'https://scholar.google.com/citations?hl=en&user=bqmEUsQAAAAJ')
1
Diana M Bowman
arizona state university
('Diana M Bowman', 'https://scholar.google.com/citations?hl=en&user=wSvWb9MAAAAJ')
1
Karen Lee Bradshaw
associate professor, rhodes university
Karen Bradshaw
arizona state university, sandra day o'connor school of law
('Karen Bradshaw', 'https://scholar.google.com/citations?hl=en&user=WzTErjUAAAAJ')
1
None
1
Andrew Carter
professor of earth sciences, birkbeck, university of london
Elizabeth J. Carter
carnegie mellon university
Dorinda Carter Andrews
associate dean for equity and inclusion and associate professor of teacher educat

In [95]:
main_df

,Name,School,Link
0,Kenneth W. Abbott,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Angela Banks,Arizona State University Sandra Day O'Connor C...,NA
0,Paul Bender,Arizona State University Sandra Day O'Connor C...,NA
0,Daniel Bodansky,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Diana Bowman,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Karen Bradshaw,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
0,Charles Calleros,Arizona State University Sandra Day O'Connor C...,NA
0,Andrew Carter,Arizona State University Sandra Day O'Connor C...,NA
0,Susan Chesler,Arizona State University Sandra Day O'Connor C...,NA
0,Adam Chodorow,Arizona State University Sandra Day O'Connor C...,https://scholar.google.com/citations?hl=en&use...
